# Credit Risk - Preprocessing

## OneHotEncoder / MinMaxScaler

In [1]:
# Imports

## Data Manipulation
import pandas as pd
import numpy as np
import pickle

# Preprocessing
from sklearn.preprocessing import StandardScaler

# Ignore Warning
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
# Package versions
%reload_ext watermark
%watermark -a "Cézar Mendes" --iversions

Author: Cézar Mendes

pandas: 1.5.3
numpy : 1.24.3
sys   : 3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]



## Loading Data

In [3]:
df = pd.read_csv('../data/credit_risk_dataset-clean.csv')

In [4]:
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [5]:
df.isna().sum()

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_status                   0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31529 entries, 0 to 31528
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  31529 non-null  int64  
 1   person_income               31529 non-null  int64  
 2   person_home_ownership       31529 non-null  object 
 3   person_emp_length           31529 non-null  float64
 4   loan_intent                 31529 non-null  object 
 5   loan_grade                  31529 non-null  object 
 6   loan_amnt                   31529 non-null  int64  
 7   loan_int_rate               31529 non-null  float64
 8   loan_status                 31529 non-null  int64  
 9   loan_percent_income         31529 non-null  float64
 10  cb_person_default_on_file   31529 non-null  object 
 11  cb_person_cred_hist_length  31529 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 2.9+ MB


## Outliers

In [7]:
num_list = df.select_dtypes(['float64', 'int64']).drop(columns = ['loan_status']).columns.tolist()
num_list

['person_age',
 'person_income',
 'person_emp_length',
 'loan_amnt',
 'loan_int_rate',
 'loan_percent_income',
 'cb_person_cred_hist_length']

### Get Overview

In [8]:
# Describe
def count_outliers(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   out = ((df < (q1 - 1.5 * IQR)) | (df > (q3 + 1.5 * IQR))).sum()
   return out

# Skew
def calc_skew(df):
    return [df[col].skew() for col in df]


# Summary Table
def getOverview(df):
    min = df.min()
    Q1 = df.quantile(0.25)
    mean = np.mean(df)
    median = df.quantile(0.5)
    Q3 = df.quantile(0.75)
    max = df.max()
    IQR = Q3 - Q1
    skew = calc_skew(df)
    outliers = count_outliers(df)
    cut_off = IQR * 1.5
    lower, upper = Q1 - cut_off, Q3 + cut_off
        
    

    new_columns = ['Columns name', 'Min', 'Q1','Mean', 'Median', 'Q3', 'Max', 'IQR', 'Lower fence', 'Upper fence', 'Skew', 'Num_Outliers']
        
    data = zip([column for column in df], min, Q1, mean,  median, Q3, max, IQR, lower, upper, skew, outliers)

    new_df = pd.DataFrame(data = data, columns = new_columns)
        
    new_df.set_index('Columns name', inplace = True)
    \
    return new_df.sort_values('Num_Outliers', ascending = False).transpose()

getOverview(df[num_list]).round(2)

Columns name,loan_amnt,person_income,person_age,cb_person_cred_hist_length,person_emp_length,loan_percent_income,loan_int_rate
Min,500.00,4000.00,20.00,2.00,0.00,0.00,5.42
Q1,5000.00,39456.00,23.00,3.00,2.00,0.09,8.49
Mean,9665.15,66704.47,27.76,5.82,4.79,0.17,11.04
Median,8000.00,56000.00,26.00,4.00,4.00,0.15,10.99
Q3,12500.00,80000.00,30.00,8.00,7.00,0.23,13.16
Max,35000.00,6000000.00,144.00,30.00,123.00,0.83,23.22
IQR,7500.00,40544.00,7.00,5.00,5.00,0.14,4.67
Lower fence,-6250.00,-21360.00,12.50,-4.50,-5.50,-0.12,1.49
Upper fence,23750.00,140816.00,40.50,15.50,14.50,0.44,20.16
Skew,1.18,33.11,2.60,1.67,2.62,1.07,0.22


#### 1 - Strategy

In [9]:
# Remove 
def remove_outliers(df, columns):
    
    for col in columns:
        Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        IQR = Q3 - Q1
        cut_off = IQR * 1.5
        lower, upper = Q1 - cut_off, Q3 + cut_off
        
        # Identify outlier rows and drop them from the original DataFrame
        df.drop(df[(df[col] > upper) | (df[col] < lower)].index, inplace=True)

remove_outliers(df, df[num_list])

In [10]:
# Check
getOverview(df[num_list]).round(2)

Columns name,person_age,person_income,loan_percent_income,loan_amnt,person_emp_length,loan_int_rate,cb_person_cred_hist_length
Min,20.00,4080.00,0.01,500.00,0.00,5.42,2.00
Q1,23.00,38400.00,0.09,4950.00,2.00,8.00,3.00
Mean,26.19,58266.96,0.16,8406.08,4.36,10.93,4.83
Median,25.00,54000.00,0.14,7500.00,4.00,10.99,4.00
Q3,28.00,74000.00,0.21,11568.75,7.00,13.06,7.00
Max,40.00,141000.00,0.41,22500.00,14.00,19.91,13.00
IQR,5.00,35600.00,0.12,6618.75,5.00,5.06,4.00
Lower fence,15.50,-15000.00,-0.09,-4978.12,-5.50,0.41,-3.00
Upper fence,35.50,127400.00,0.39,21496.88,14.50,20.65,13.00
Skew,0.98,0.77,0.67,0.71,0.67,0.15,0.93


#### 2 - Strategy

In [11]:
# Replace Outliers
def replace_outliers_with_fences(df, columns):
    
    for col in columns:
        Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        IQR = Q3 - Q1
        cut_off = IQR * 1.5
        lower, upper = Q1 - cut_off, Q3 + cut_off

        # Replace outlier values with the respective fences
        df[col] = np.where(df[col] > upper, upper, df[col])
        df[col] = np.where(df[col] < lower, lower, df[col])

In [12]:
replace_outliers_with_fences(df,df[num_list])

In [13]:
# Check again!
getOverview(df[num_list]).round(2)

Columns name,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length
Min,20.00,4080.00,0.00,500.00,5.42,0.01,2.00
Q1,23.00,38400.00,2.00,4950.00,8.00,0.09,3.00
Mean,26.14,58154.03,4.36,8404.08,10.93,0.16,4.83
Median,25.00,54000.00,4.00,7500.00,10.99,0.14,4.00
Q3,28.00,74000.00,7.00,11568.75,13.06,0.21,7.00
Max,35.50,127400.00,14.00,21496.88,19.91,0.39,13.00
IQR,5.00,35600.00,5.00,6618.75,5.06,0.12,4.00
Lower fence,15.50,-15000.00,-5.50,-4978.12,0.41,-0.09,-3.00
Upper fence,35.50,127400.00,14.50,21496.88,20.65,0.39,13.00
Skew,0.81,0.71,0.67,0.71,0.15,0.65,0.93


In [14]:
df.isnull().sum()

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_status                   0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64

# Saving Dataset After outliers

In [15]:
# Save the DataFrame to a CSV file
df.to_csv('../data/credit_risk_dataset-clean-out.csv', index=False)

## Encoding

In [16]:
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
1,21.0,9600.0,OWN,5.0,EDUCATION,B,1000.0,11.14,0,0.10,N,2.0
5,21.0,9900.0,OWN,2.0,VENTURE,A,2500.0,7.14,1,0.25,N,2.0
9,21.0,10000.0,OWN,6.0,VENTURE,D,1600.0,14.74,1,0.16,N,3.0
19,24.0,10800.0,MORTGAGE,8.0,EDUCATION,B,1750.0,10.99,1,0.16,N,2.0
23,24.0,10980.0,OWN,0.0,PERSONAL,A,1500.0,7.29,0,0.14,N,3.0


In [17]:
# Checking Data
df["person_home_ownership"].unique().tolist()

['OWN', 'MORTGAGE', 'OTHER', 'RENT']

In [18]:
df["loan_intent"].unique().tolist()

['EDUCATION',
 'VENTURE',
 'PERSONAL',
 'MEDICAL',
 'DEBTCONSOLIDATION',
 'HOMEIMPROVEMENT']

In [19]:
df["loan_grade"].unique().tolist()

['B', 'A', 'D', 'C', 'E', 'F', 'G']

In [20]:
df["cb_person_default_on_file"].unique().tolist()

['N', 'Y']

### Target Encoding

In [21]:
df["cb_person_default_on_file"] = np.where(df["cb_person_default_on_file"] == 'Y', 1, 0)
df.sample(5)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
19541,30.0,39996.0,MORTGAGE,2.0,PERSONAL,B,10000.0,10.75,0,0.25,0,8.0
8908,22.0,22800.0,RENT,4.0,EDUCATION,C,5000.0,13.85,0,0.22,1,2.0
19894,30.0,93500.0,RENT,3.0,EDUCATION,C,9925.0,12.09,0,0.11,1,7.0
5548,22.0,69108.0,RENT,2.0,PERSONAL,E,5400.0,15.95,1,0.08,1,2.0
17922,34.0,72000.0,RENT,7.0,EDUCATION,D,18000.0,15.95,0,0.25,1,8.0


### OneHotEncoder

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25334 entries, 1 to 31245
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  25334 non-null  float64
 1   person_income               25334 non-null  float64
 2   person_home_ownership       25334 non-null  object 
 3   person_emp_length           25334 non-null  float64
 4   loan_intent                 25334 non-null  object 
 5   loan_grade                  25334 non-null  object 
 6   loan_amnt                   25334 non-null  float64
 7   loan_int_rate               25334 non-null  float64
 8   loan_status                 25334 non-null  int64  
 9   loan_percent_income         25334 non-null  float64
 10  cb_person_default_on_file   25334 non-null  int32  
 11  cb_person_cred_hist_length  25334 non-null  float64
dtypes: float64(7), int32(1), int64(1), object(3)
memory usage: 2.4+ MB


In [23]:
# Selecting the categorical features
categorical_features = ['person_home_ownership','loan_intent', 'loan_grade']

# Applying one-hot encoding to the categorical features
df = pd.get_dummies(df, columns=categorical_features)

# Displaying the first few rows of the one-hot encoded data
df.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,...,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G
1,21.0,9600.0,5.0,1000.0,11.14,0,0.10,0,2.0,0,...,0,0,0,0,1,0,0,0,0,0
5,21.0,9900.0,2.0,2500.0,7.14,1,0.25,0,2.0,0,...,0,0,1,1,0,0,0,0,0,0
9,21.0,10000.0,6.0,1600.0,14.74,1,0.16,0,3.0,0,...,0,0,1,0,0,0,1,0,0,0
19,24.0,10800.0,8.0,1750.0,10.99,1,0.16,0,2.0,1,...,0,0,0,0,1,0,0,0,0,0
23,24.0,10980.0,0.0,1500.0,7.29,0,0.14,0,3.0,0,...,0,1,0,1,0,0,0,0,0,0


In [24]:
encoded_data = df.copy()

In [25]:
max(df['loan_percent_income'])

0.39

In [26]:
df.isnull().sum()

person_age                        0
person_income                     0
person_emp_length                 0
loan_amnt                         0
loan_int_rate                     0
loan_status                       0
loan_percent_income               0
cb_person_default_on_file         0
cb_person_cred_hist_length        0
person_home_ownership_MORTGAGE    0
person_home_ownership_OTHER       0
person_home_ownership_OWN         0
person_home_ownership_RENT        0
loan_intent_DEBTCONSOLIDATION     0
loan_intent_EDUCATION             0
loan_intent_HOMEIMPROVEMENT       0
loan_intent_MEDICAL               0
loan_intent_PERSONAL              0
loan_intent_VENTURE               0
loan_grade_A                      0
loan_grade_B                      0
loan_grade_C                      0
loan_grade_D                      0
loan_grade_E                      0
loan_grade_F                      0
loan_grade_G                      0
dtype: int64

## StandarScaler

In [27]:
columns_to_scale = ["person_age", "person_income",
                    "person_emp_length", "loan_amnt", "loan_int_rate", "cb_person_cred_hist_length"]

In [28]:
scaler = StandardScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

df.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,...,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G
1,-1.343141,-1.842523,0.191132,-1.536690,0.070196,0,0.10,0,-1.045831,0,...,0,0,0,0,1,0,0,0,0,0
5,-1.343141,-1.831139,-0.698696,-1.225370,-1.256310,1,0.25,0,-1.045831,0,...,0,0,1,1,0,0,0,0,0,0
9,-1.343141,-1.827344,0.487742,-1.412162,1.264051,1,0.16,0,-0.676474,0,...,0,0,1,0,0,0,1,0,0,0
19,-0.558837,-1.796986,1.080960,-1.381030,0.020452,1,0.16,0,-1.045831,1,...,0,0,0,0,1,0,0,0,0,0
23,-0.558837,-1.790155,-1.291914,-1.432917,-1.206566,0,0.14,0,-0.676474,0,...,0,1,0,1,0,0,0,0,0,0


## MinMaxScaler

# Saving Objects

In [29]:
df.to_csv("../data/credit_risk-processed.csv", index=False)

In [30]:
# saving the scaler
pickle.dump(scaler, open("../object/scaler.pkl", "wb"))

In [31]:
# Define the path to save the pickle file
pickle_path = "../object/encoder.pkl"

# Save the one-hot encoded DataFrame as a pickle file
encoded_data.to_pickle(pickle_path)

pickle_path


'../object/encoder.pkl'